# TO GRANT OR NOT TO GRANT: DECIDING ON COMPENSATION BENEFITS - PART 1: INITIAL INSPECTION

## 1. Imports, options and ingestion

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import timedelta
from matplotlib import pyplot as plt
import seaborn as sns
import re
import math

In [2]:
# setting the options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)
plt.style.use('seaborn-v0_8-dark')

In [3]:
# importing the training and test data
df = pd.read_csv('../Data/train_data.csv', sep=',')
df_test = pd.read_csv('../Data/test_data.csv', sep=',')


/tmp/ipykernel_6560/858450308.py:2: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/train_data.csv', sep=',')


Just by importing the dataset, we get a warning saying column 29 has mixed data types - we will check this in a bit.

## 2. Macro-inspection

In [4]:
# we check the shape of the dataset
df.shape

(593471, 33)

In [5]:
# we check the first rows of the dataset
df.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Identifier,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,5393875,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,5393091,4. TEMPORARY,WYOMING,N,ROCHESTER,2020-02-21,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,5393889,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIAT,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957648180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,2A. SIF,5393887,2. NON-COMP,DUTCHESS,N,ALBANY,NaN,M,NaN,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,NaN,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [6]:
# we check if there are any aggregation rows at the end of the dataset
df.tail()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Identifier,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
593466,NaN,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327160035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593467,2022-12-13,72.0,N,2022-12-31,N,0.0,1950.0,2022-12-31,NaN,TECHNOLOGY INSURANCE CO. INC.,1A. PRIVATE,6165075,2. NON-COMP,SULLIVAN,N,BINGHAMTON,NaN,F,NaN,48.0,TRANSPORTATION AND WAREHOUSING,I,NaN,25.0,FROM DIFFERENT LEVEL (ELEVATION),90.0,MULTIPLE PHYSICAL INJURIES ONLY,-9.0,MULTIPLE,12779,0.0,Not Work Related,3.0
593468,NaN,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249875936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593469,NaN,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120584215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593470,NaN,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,818961390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# we check the datatypes and null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593471 entries, 0 to 593470
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       570337 non-null  object 
 1   Age at Injury                       574026 non-null  float64
 2   Alternative Dispute Resolution      574026 non-null  object 
 3   Assembly Date                       593471 non-null  object 
 4   Attorney/Representative             574026 non-null  object 
 5   Average Weekly Wage                 545375 non-null  float64
 6   Birth Year                          544948 non-null  float64
 7   C-2 Date                            559466 non-null  object 
 8   C-3 Date                            187245 non-null  object 
 9   Carrier Name                        574026 non-null  object 
 10  Carrier Type                        574026 non-null  object 
 11  Claim Identifier          

**Data type analysis:**

Features that should be integers:
- 'Age at Injury'
- 'Birth Year'
- 'IME-4 Count'
- 'Industry Code'
- 'WCIO Cause of Injury Code'
- 'WCIO Nature of Injury Code'
- 'WCIO Part Of Body Code'
- 'Number of Dependents'

Features that should be booleans:
- 'Agreement Reached'

### 2.1. Claim Identifier

This feature is the unique identifier of each claim - we will analyse this column in more depth in an attempt to assign it as the index of our dataframe.

In [8]:
# we check if there are any duplicate values for this column
df[df['Claim Identifier'].duplicated(keep=False)]

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Identifier,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
257901,NaN,NaN,NaN,2021-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292668076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
526445,NaN,NaN,NaN,2022-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292668076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Apparently, we have one duplicated 'Claim Identifier', where all values, except for 'Assembly Date', are NaNs. </br>
We will use the default python behavior and drop the second appearence of the repeated 'Claim Identifier'. </br>
As we saw previously, this column has no null values, so we can set it as the dataframe index.

In [9]:
df = df[~df['Claim Identifier'].duplicated()].set_index('Claim Identifier').rename_axis(None)

In [10]:
df.head(3)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
5393875,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,2020-02-21,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIAT,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0


## 3. Missing Values

In [11]:
# we calculate the number of NaNs for each column
nan_counts = df.isna().sum()

# we get the total number of rows
total_rows = df.shape[0]

# we calculate the percentage of NaN values for each column
percentage_nans = (nan_counts / total_rows) * 100

# we format the percentages with '%' sign
percentage_nans = percentage_nans.apply(lambda x: f"{x:.2f}%")

# we combine all the information into a DataFrame
nan_summary = pd.DataFrame({
    'NaN Count': nan_counts
    , 'Total Values': [total_rows] * len(nan_counts)
    , 'Percentage NaN': percentage_nans
})

display(nan_summary)

,NaN Count,Total Values,Percentage NaN
Accident Date,23133,593470,3.90%
Age at Injury,19444,593470,3.28%
Alternative Dispute Resolution,19444,593470,3.28%
Assembly Date,0,593470,0.00%
Attorney/Representative,19444,593470,3.28%
Average Weekly Wage,48095,593470,8.10%
Birth Year,48522,593470,8.18%
C-2 Date,34004,593470,5.73%
C-3 Date,406225,593470,68.45%
Carrier Name,19444,593470,3.28%


By analysing the output above, we conclude:
- **'C-3 Date'**: more than 50% of the data for this feature is missing, it can be due to process status - the employee has not yet sent the report
- **'First Hearing Date'**: around 75% of the data for this feature is missing - this means hearings have not been held
- **'IME-4 Count'**: more than 75% of the data for this feature is missing, it can be due to process status - the independent examiner has not sent the report
- **'OIICS Nature of Injury Description'**: only has null values

## 4. Data types

As we imported the data, we got a warning saying columns had mixed data types. We shall take a look at this issue now.

In [12]:
def check_mixed_types(column):
    # if there are more than one unique data types (we exclude NaN values, as these are considered floats)
    return len(set(column[pd.notna(column)].apply(type))) > 1

In [13]:
mixed_type_columns = [col for col in df.columns if check_mixed_types(df[col])]

print(mixed_type_columns)

['Zip Code']


In [14]:
for col in mixed_type_columns:
    print(df[col].apply(type).value_counts())

Zip Code
<class 'str'>      530364
<class 'float'>     63106
Name: count, dtype: int64


The values for 'Zip Code' are split across two data types - string and float. We shall look at the feature values in order to make a decision.

In [15]:
# we check if there are any values that contain any non numeric character to make a decision about the final data type
print(df[df['Zip Code'].apply(lambda x: bool(re.search(r'\D', str(x))))]['Zip Code'].unique())

[nan 'L1N 5' 'T1B0P' 'L6Y 1' 'JMDMR' 'N2P 1' 'H7X3Z' 'L2N 3' 'L0R 1'
 'L1A 3' 'T4R1E' 'L7L 6' 'L2W 1' 'K9K 1' '.1605' 'M4E1S' 'L2A 5' 'UNKNO'
 'T2V' 'J0L1B' 'AB106' 'J1H2Y' 'L2A 1' 'H2T2W' 'T1J5G' '22-40' 'M3K2B'
 'M3K2C' 'T1S 0' 'N6C4E' 'H1V2L' 'V6Z3G' 'KOC 1' 'M6S5B' 'V3H1H' 'L2H0H'
 'L1C0K' 'JOL1B' 'SLV' 'BT48-' 'L5N7C' 'JMDCN' 'V6T1Z' 'T2X3P' 'L2A1R'
 'L4A0G' 'H1V3S' 'K6H2L' 'JMAKN' 'L6H 3' 'J0J1S' 'B3E1A' 'V6T5C' 'JMBTS'
 'V2B0E' 'T6R 0' 'JMDWD' 'M6K3C' 'L9W' 'S7R 0' 'F94A0' 'P7G' 'Z2E1N'
 'L0S1N' 'K2L 4' 'H1G0A' 'B3M 3' 10567.0 11213.0 11421.0 11378.0 10035.0
 11385.0 10302.0 10305.0 12550.0 10027.0 11710.0 12180.0 11365.0 11418.0
 10598.0 10465.0 18851.0 6820.0 10952.0 14557.0 72524.0 7882.0 10923.0
 12901.0 11207.0 12590.0 11580.0 11725.0 10033.0 10532.0 12804.0 14614.0
 11779.0 11224.0 11435.0 11735.0 11367.0 11772.0 11218.0 11434.0 11901.0
 11577.0 11004.0 13083.0 10037.0 11214.0 10458.0 11422.0 14590.0 7822.0
 10001.0 10306.0 11704.0 10453.0 11414.0 11206.0 11203.0 11212.0 1

Since we have determined that there are values for 'Zip Code' that contain other than numeric characters, we shall set this feature as a string.

In [16]:
# we change the data type to string
df['Zip Code'] = df['Zip Code'].astype(str)

In [17]:
# re-checking the data type counts
print(df['Zip Code'].apply(type).value_counts())

Zip Code
<class 'str'>    593470
Name: count, dtype: int64


Let us now fixed the remaining data types, as identified in [Section 2](#2-macro-inspection)

In [18]:
features_to_cast = [
    'Age at Injury'
    , 'Birth Year'
    , 'IME-4 Count'
    , 'Industry Code'
    , 'WCIO Cause of Injury Code'
    , 'WCIO Nature of Injury Code'
    , 'WCIO Part Of Body Code'
    , 'Number of Dependents'
    , 'Agreement Reached'
]

In [19]:
for feature in features_to_cast:
    df[feature] = pd.to_numeric(df[feature], errors='coerce').astype('Int64')

In [20]:
# checking if the conversion was successful
for feature in features_to_cast:
    print(feature, '\t', df[feature].dtypes)

Age at Injury 	 Int64
Birth Year 	 Int64
IME-4 Count 	 Int64
Industry Code 	 Int64
WCIO Cause of Injury Code 	 Int64
WCIO Nature of Injury Code 	 Int64
WCIO Part Of Body Code 	 Int64
Number of Dependents 	 Int64
Agreement Reached 	 Int64


## 5. Date formats

In [21]:
date_columns = [
    'Accident Date'
    , 'Assembly Date'
    , 'C-2 Date'
    , 'C-3 Date'
    , 'First Hearing Date'
]

In [22]:
def check_date_yyyymmdd(df, col):
    date_pattern = r'^(\d{4})-(\d{2})-(\d{2})$'

    def validate(date):
        if isinstance(date, str):
        # we check if the date is in the correct format
            match = re.match(date_pattern, date)

            # if it matches the format
            if match:

                # define month and day
                month = int(match.group(1))
                day = int(match.group(2))

                # we check if month is between 1 and 12
                if 1 <= month <= 12:

                    # we check if the months have the correct max day
                    if (month == 2 and day <= 29) or \
                    (month in [4, 6, 9, 11] and day <= 30) or \
                    (month in [1, 3, 5, 7, 8, 10, 12] and day <= 31):
                        
                        return True
                    
            return False
    
    return df[col].apply(validate)

In [23]:
# check if there are any values in these date columns that do not follow this format
for column in date_columns:
    if check_date_yyyymmdd(df, column).any():
        display(check_date_yyyymmdd(df, column))

Since we have assessed that all dates are in the YYYY-MM-DD format, we are going to transform these columns into datetime

In [24]:
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d')

In [25]:
df[date_columns].dtypes

Accident Date         datetime64[ns]
Assembly Date         datetime64[ns]
C-2 Date              datetime64[ns]
C-3 Date              datetime64[ns]
First Hearing Date    datetime64[ns]
dtype: object

In [26]:
# TODO: Convert to ordinal

## 6. Unique values

In [27]:
# we check the number of unique values for each column
df.nunique()

Accident Date                           5539
Age at Injury                            108
Alternative Dispute Resolution             3
Assembly Date                           1096
Attorney/Representative                    2
Average Weekly Wage                   120024
Birth Year                               107
C-2 Date                                2475
C-3 Date                                1648
Carrier Name                            2046
Carrier Type                               8
Claim Injury Type                          8
County of Injury                          63
COVID-19 Indicator                         2
District Name                              8
First Hearing Date                      1094
Gender                                     4
IME-4 Count                               41
Industry Code                             24
Industry Code Description                 20
Medical Fee Region                         5
OIICS Nature of Injury Description         0
WCIO Cause

In [28]:
# we check the unique values for each column
pd.DataFrame(
    {
        'feature_type': [df[column].dtype for column in df.columns],
        'unique_values': df.apply(lambda col: sorted(pd.Series(col.dropna().unique().tolist())))  # we disregard NaN values, so we can sort the unique values
    }
)

,feature_type,unique_values
Accident Date,datetime64[ns],"[1961-09-06 00:00:00, 1963-10-01 00:00:00, 1966-06-01 00:00:00, 1966-09-13 00:00:00, 1967-01-01 00:00:00, 1967-04-05 00:00:00, 1967-10-26 00:00:00, 1969-04-01 00:00:00, 1969-05-03 00:00:00, 1970-12-10 00:00:00, 1971-03-17 00:00:00, 1971-04-27 00:00:00, 1971-05-08 00:00:00, 1972-05-26 00:00:00, 1972-09-26 00:00:00, 1972-11-15 00:00:00, 1972-12-26 00:00:00, 1973-01-15 00:00:00, 1973-02-27 00:00:00, 1973-04-24 00:00:00, 1973-05-01 00:00:00, 1973-05-31 00:00:00, 1973-12-12 00:00:00, 1973-12-24 00:00:00, 1974-02-22 00:00:00, 1974-06-06 00:00:00, 1974-06-22 00:00:00, 1974-07-09 00:00:00, 1974-10-29 00:00:00, 1975-05-01 00:00:00, 1975-05-06 00:00:00, 1975-06-09 00:00:00, 1975-06-17 00:00:00, 1975-07-01 00:00:00, 1975-08-06 00:00:00, 1975-10-16 00:00:00, 1975-12-22 00:00:00, 1976-07-21 00:00:00, 1976-08-17 00:00:00, 1976-10-14 00:00:00, 1977-03-04 00:00:00, 1977-04-04 00:00:00, 1977-06-15 00:00:00, 1977-06-17 00:00:00, 1977-06-21 00:00:00, 1977-07-22 00:00:00, 1977-09-16 00:00:00, 1977-12-29 00:00:00, 1978-01-23 00:00:00, 1978-02-02 00:00:00, 1978-04-04 00:00:00, 1978-05-19 00:00:00, 1978-07-28 00:00:00, 1978-08-14 00:00:00, 1979-03-01 00:00:00, 1979-08-28 00:00:00, 1979-10-08 00:00:00, 1979-11-02 00:00:00, 1980-01-01 00:00:00, 1980-01-16 00:00:00, 1980-03-26 00:00:00, 1980-03-28 00:00:00, 1980-04-24 00:00:00, 1980-04-28 00:00:00, 1980-05-01 00:00:00, 1980-05-09 00:00:00, 1980-05-17 00:00:00, 1980-10-06 00:00:00, 1980-12-16 00:00:00, 1980-12-17 00:00:00, 1981-01-28 00:00:00, 1981-02-09 00:00:00, 1981-02-16 00:00:00, 1981-04-21 00:00:00, 1981-06-22 00:00:00, 1981-09-23 00:00:00, 1981-11-12 00:00:00, 1981-12-09 00:00:00, 1981-12-30 00:00:00, 1982-01-01 00:00:00, 1982-02-01 00:00:00, 1982-03-12 00:00:00, 1982-03-25 00:00:00, 1982-05-21 00:00:00, 1982-06-10 00:00:00, 1982-06-14 00:00:00, 1982-08-01 00:00:00, 1982-10-15 00:00:00, 1982-12-07 00:00:00, 1983-01-18 00:00:00, 1983-02-14 00:00:00, 1983-02-19 00:00:00, 1983-05-01 00:00:00, 1983-06-06 00:00:00, 1983-07-12 00:00:00, 1983-07-18 00:00:00, 1983-07-28 00:00:00, 1983-08-22 00:00:00, 1984-01-16 00:00:00, 1984-01-30 00:00:00, ...]"
Age at Injury,Int64,"[0, 1, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, ...]"
Alternative Dispute Resolution,object,"[N, U, Y]"
Assembly Date,datetime64[ns],"[2020-01-01 00:00:00, 2020-01-02 00:00:00, 2020-01-03 00:00:00, 2020-01-04 00:00:00, 2020-01-05 00:00:00, 2020-01-06 00:00:00, 2020-01-07 00:00:00, 2020-01-08 00:00:00, 2020-01-09 00:00:00, 2020-01-10 00:00:00, 2020-01-11 00:00:00, 2020-01-12 00:00:00, 2020-01-13 00:00:00, 2020-01-14 00:00:00, 2020-01-15 00:00:00, 2020-01-16 00:00:00, 2020-01-17 00:00:00, 2020-01-18 00:00:00, 2020-01-19 00:00:00, 2020-01-20 00:00:00, 2020-01-21 00:00:00, 2020-01-22 00:00:00, 2020-01-23 00:00:00, 2020-01-24 00:00:00, 2020-01-25 00:00:00, 2020-01-26 00:00:00, 2020-01-27 00:00:00, 2020-01-28 00:00:00, 2020-01-29 00:00:00, 2020-01-30 00:00:00, 2020-01-31 00:00:00, 2020-02-01 00:00:00, 2020-02-02 00:00:00, 2020-02-03 00:00:00, 2020-02-04 00:00:00, 2020-02-05 00:00:00, 2020-02-06 00:00:00, 2020-02-07 00:00:00, 2020-02-08 00:00:00, 2020-02-09 00:00:00, 2020-02-10 00:00:00, 2020-02-11 00:00:00, 2020-02-12 00:00:00, 2020-02-13 00:00:00, 2020-02-14 00:00:00, 2020-02-15 00:00:00, 2020-02-16 00:00:00, 2020-02-17 00:00:00, 2020-02-18 00:00:00, 2020-02-19 00:00:00, 2020-02-20 00:00:00, 2020-02-21 00:00:00, 2020-02-22 00:00:00, 2020-02-23 00:00:00, 2020-02-24 00:00:00, 2020-02-25 00:00:00, 2020-02-26 00:00:00, 2020-02-27 00:00:00, 2020-02-28 00:00:00, 2020-02-29 00:00:00, 2020-03-01 00:00:00, 2020-03-02 00:00:00, 2020-03-03 00:00:00, 2020-

Looking at this information, we can point out the following:
- **Age at Injury**: since this dataset concerns workplace injuries, values under 14 (legal working age in the US) are weird
- **Birth Year**: we have '0.0' values
- **OIICS Nature of Injury Description**: has no values
- **WCIO Part Of Body Code**: has a negative value
- **WCB Decision**: only has one value

## 7. Data consistency

### 7.1. Codes vs. Descriptions

In this section, we will check if the amount of codes are the same as the descriptions.

<span style="color:darkred">**Note to future us:**</span> We may want to aggregate some more based on description similarities.

#### 7.1.1. Industry

In [29]:
len(df['Industry Code'].dropna().unique()) == len(df['Industry Code Description'].dropna().unique())

False

In [30]:
df.groupby(['Industry Code', 'Industry Code Description']).size().reset_index(name='count').sort_values('Industry Code Description').reset_index(drop=True)

,Industry Code,Industry Code Description,count
0,72,ACCOMMODATION AND FOOD SERVICES,26456
1,56,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIAT,21027
2,11,"AGRICULTURE, FORESTRY, FISHING AND HUNTING",2404
3,71,"ARTS, ENTERTAINMENT, AND RECREATION",9967
4,23,CONSTRUCTION,30903
5,61,EDUCATIONAL SERVICES,44393
6,52,FINANCE AND INSURANCE,8214
7,62,HEALTH CARE AND SOCIAL ASSISTANCE,114339
8,51,INFORMATION,9166
9,55,MANAGEMENT OF COMPANIES AND ENTERPRISES,370


The "repeated" values of 'Industry Code Description' occur for consecutive 'Industry Code' values - we will aggregate the codes into the lower valued one

In [31]:
# group by 'Industry Code Description' and get the minimum 'Industry Code'
min_codes = df.groupby('Industry Code Description')['Industry Code'].min().reset_index()
min_codes.rename(columns={'Industry Code': 'Min Industry Code'}, inplace=True)

# add the minimum codes to the dataframe
df = df.merge(min_codes, on='Industry Code Description', how='left')

# replace 'Industry Code' with 'Min Industry Code'
df['Industry Code'] = df['Min Industry Code']

# drop the auxiliary column
df.drop(columns=['Min Industry Code'], inplace=True)

In [32]:
# we check if the changes were successful
df.groupby(['Industry Code', 'Industry Code Description']).size().reset_index(name='count').sort_values('Industry Code Description').reset_index(drop=True)

,Industry Code,Industry Code Description,count
0,72,ACCOMMODATION AND FOOD SERVICES,26456
1,56,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIAT,21027
2,11,"AGRICULTURE, FORESTRY, FISHING AND HUNTING",2404
3,71,"ARTS, ENTERTAINMENT, AND RECREATION",9967
4,23,CONSTRUCTION,30903
5,61,EDUCATIONAL SERVICES,44393
6,52,FINANCE AND INSURANCE,8214
7,62,HEALTH CARE AND SOCIAL ASSISTANCE,114339
8,51,INFORMATION,9166
9,55,MANAGEMENT OF COMPANIES AND ENTERPRISES,370


#### 7.1.2. WCIO Cause of Injury

In [33]:
len(df['WCIO Cause of Injury Code'].dropna().unique()) == len(df['WCIO Cause of Injury Description'].dropna().unique())

False

In [34]:
df.groupby(['WCIO Cause of Injury Code', 'WCIO Cause of Injury Description']).size().reset_index(name='count').sort_values('WCIO Cause of Injury Description').reset_index(drop=True)

,WCIO Cause of Injury Code,WCIO Cause of Injury Description,count
0,14,ABNORMAL AIR PRESSURE,51
1,82,"ABSORPTION, INGESTION OR INHALATION, NOC",3189
2,85,ANIMAL OR INSECT,6502
3,15,BROKEN GLASS,2385
4,13,"CAUGHT IN, UNDER OR BETWEEN, NOC",7212
5,1,CHEMICALS,1863
6,11,COLD OBJECTS OR SUBSTANCES,199
7,20,COLLAPSING MATERIALS (SLIDES OF EARTH),567
8,45,COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE,16826
9,46,COLLISION WITH A FIXED OBJECT,1453


Again, we will aggregate the same value for the description in the same code.

In [35]:
# group by 'WCIO Cause of Injury Description' and get the minimum 'WCIO Cause of Injury Code'
min_codes = df.groupby('WCIO Cause of Injury Description')['WCIO Cause of Injury Code'].min().reset_index()
min_codes.rename(columns={'WCIO Cause of Injury Code': 'Min WCIO Cause of Injury Code'}, inplace=True)

# add the minimum codes to the dataframe
df = df.merge(min_codes, on='WCIO Cause of Injury Description', how='left')

# replace 'WCIO Cause of Injury Code' with 'Min WCIO Cause of Injury Code'
df['WCIO Cause of Injury Code'] = df['Min WCIO Cause of Injury Code']

# drop the auxiliary column
df.drop(columns=['Min WCIO Cause of Injury Code'], inplace=True)

In [36]:
df.groupby(['WCIO Cause of Injury Code', 'WCIO Cause of Injury Description']).size().reset_index(name='count').sort_values('WCIO Cause of Injury Description').reset_index(drop=True)

,WCIO Cause of Injury Code,WCIO Cause of Injury Description,count
0,14,ABNORMAL AIR PRESSURE,51
1,82,"ABSORPTION, INGESTION OR INHALATION, NOC",3189
2,85,ANIMAL OR INSECT,6502
3,15,BROKEN GLASS,2385
4,13,"CAUGHT IN, UNDER OR BETWEEN, NOC",7212
5,1,CHEMICALS,1863
6,11,COLD OBJECTS OR SUBSTANCES,199
7,20,COLLAPSING MATERIALS (SLIDES OF EARTH),567
8,45,COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE,16826
9,46,COLLISION WITH A FIXED OBJECT,1453


There is also some values that are rather similiar that could be aggregated (e.g. 'MOVING PART OF MACHINE' and 'MOVING PARTS OF MACHINE')

#### 7.1.3. WCIO Nature of Injury

In [37]:
len(df['WCIO Nature of Injury Code'].dropna().unique()) == len(df['WCIO Nature of Injury Description'].dropna().unique())

True

In [38]:
df.groupby(['WCIO Nature of Injury Code', 'WCIO Nature of Injury Description']).size().reset_index(name='count').sort_values('WCIO Nature of Injury Description').reset_index(drop=True)

,WCIO Nature of Injury Code,WCIO Nature of Injury Description,count
0,38,ADVERSE REACTION TO A VACCINATION OR INOCULATION,185
1,75,AIDS,32
2,80,"ALL OTHER CUMULATIVE INJURY, NOC",2798
3,71,"ALL OTHER OCCUPATIONAL DISEASE INJURY, NOC",1917
4,59,"ALL OTHER SPECIFIC INJURIES, NOC",36101
5,2,AMPUTATION,736
6,3,ANGINA PECTORIS,181
7,61,ASBESTOSIS,112
8,54,ASPHYXIATION,94
9,62,BLACK LUNG,5


#### 7.1.4. WCIO Part of Body

In [39]:
len(df['WCIO Part Of Body Code'].dropna().unique()) == len(df['WCIO Part Of Body Description'].dropna().unique())

False

In [40]:
df.groupby(['WCIO Part Of Body Code', 'WCIO Part Of Body Description']).size().reset_index(name='count').sort_values('WCIO Part Of Body Description').reset_index(drop=True)

,WCIO Part Of Body Code,WCIO Part Of Body Description,count
0,61,ABDOMEN INCLUDING GROIN,6770
1,55,ANKLE,24928
2,64,ARTIFICIAL APPLIANCE,53
3,91,BODY SYSTEMS AND MULTIPLE BODY SYSTEMS,10950
4,12,BRAIN,1989
5,62,BUTTOCKS,1161
6,44,CHEST,7644
7,22,DISC,1103
8,43,DISC,1893
9,13,EAR(S),3912


Once again, we have descriptions associated to multiple codes - we can aggregate these descriptions into one single code.

In [41]:
# group by 'WCIO Part Of Body Description' and get the minimum 'WCIO Part Of Body Code'
min_codes = df.groupby('WCIO Part Of Body Description')['WCIO Part Of Body Code'].min().reset_index()
min_codes.rename(columns={'WCIO Part Of Body Code': 'Min WCIO Part Of Body Code'}, inplace=True)

# add the minimum codes to the dataframe
df = df.merge(min_codes, on='WCIO Part Of Body Description', how='left')

# replace 'WCIO Cause of Injury Code' with 'Min WCIO Cause of Injury Code'
df['WCIO Part Of Body Code'] = df['Min WCIO Part Of Body Code']

# drop the auxiliary column
df.drop(columns=['Min WCIO Part Of Body Code'], inplace=True)

In [42]:
df.groupby(['WCIO Part Of Body Code', 'WCIO Part Of Body Description']).size().reset_index(name='count').sort_values(by='WCIO Part Of Body Description').reset_index(drop=True)

,WCIO Part Of Body Code,WCIO Part Of Body Description,count
0,61,ABDOMEN INCLUDING GROIN,6770
1,55,ANKLE,24928
2,64,ARTIFICIAL APPLIANCE,53
3,91,BODY SYSTEMS AND MULTIPLE BODY SYSTEMS,10950
4,12,BRAIN,1989
5,62,BUTTOCKS,1161
6,44,CHEST,7644
7,22,DISC,2996
8,13,EAR(S),3912
9,32,ELBOW,9949


### 7.2. Birth Year

We will try to change the 0 values by computing a new value using 'Accident Date' and 'Age at Injury'

In [43]:
# amount of 0 values
len(df[df['Birth Year'] == 0])

25081

In [44]:
# amount of values we can fix using valid values of the other two columns
len(df[(df['Birth Year'] == 0) & (df['Accident Date'].notna()) & (df['Age at Injury'] >= 14)])

23141

In [45]:
# we define the update condition
condition = (df['Birth Year'] == 0) & (df['Accident Date'].notna()) & (df['Age at Injury'] >= 14)

# we update the values based on the condition
df.loc[condition, 'Birth Year'] = df['Accident Date'].dt.year - df['Age at Injury']

In [46]:
# amount of 0 values after the changes
len(df[df['Birth Year'] == 0])

1940

We will drop these rows as they are inconsistent with the data.

In [47]:
df = df[(df['Birth Year'] != 0) | (df['Birth Year'].isna())]

### 7.3. Age at Injury

We will check the values that are under the minimum legal working age.

In [48]:
len(df[df['Age at Injury'] < 14])

3571

In [49]:
# amount of values we will be able to correct using 'Accident Date' and 'Birth Year'
len(df[(df['Age at Injury'] < 14) & (df['Accident Date'].notna()) & (df['Birth Year'] != 0)])

122

In [50]:
age_condition = (df['Age at Injury'] < 14) & (df['Accident Date'].notna()) & (df['Birth Year'] != 0)

df.loc[age_condition, 'Age at Injury'] = df['Accident Date'].dt.year - df['Birth Year']

In [51]:
# amount of values that remained inconsistent
len(df[(df['Age at Injury'] < 14) & (df['Accident Date'].notna()) & (df['Birth Year'] != 0)])

27

In [52]:
# amount of values we will be able to compute after imputation of missing values
len(df[(df['Age at Injury'] < 14) & ((df['Accident Date'].isna()) | (df['Birth Year'].isna()))])

3449

In [53]:
# amount of values we cannot fix
len(df[(df['Age at Injury'] < 14) & ~((df['Accident Date'].isna()) | (df['Birth Year'].isna()))])

27

We shall remove the "unfixable" values.

In [54]:
condition = (df['Age at Injury'] < 14) & ~((df['Accident Date'].isna()) | (df['Birth Year'].isna()))

df = df.loc[~condition]

### 7.4. Gender

In [55]:
df['Gender'].value_counts()

Gender
M    333897
F    233456
U      4660
X        46
Name: count, dtype: int64

In [56]:
df.groupby(['Gender', 'Claim Injury Type'])['Claim Injury Type'].count()

Gender  Claim Injury Type
F       1. CANCELLED           3962
        2. NON-COMP          128927
        3. MED ONLY           28084
        4. TEMPORARY          56088
        5. PPD SCH LOSS       14621
        6. PPD NSL             1691
        7. PTD                   23
        8. DEATH                 60
M       1. CANCELLED           7746
        2. NON-COMP          158122
        3. MED ONLY           40244
        4. TEMPORARY          91344
        5. PPD SCH LOSS       33454
        6. PPD NSL             2506
        7. PTD                   74
        8. DEATH                407
U       1. CANCELLED            159
        2. NON-COMP            3055
        3. MED ONLY             450
        4. TEMPORARY            804
        5. PPD SCH LOSS         179
        6. PPD NSL               13
X       1. CANCELLED              3
        2. NON-COMP              12
        3. MED ONLY              18
        4. TEMPORARY              5
        5. PPD SCH LOSS           8
Na

Looking at these catories and after a little bit of research, we can assume that F, M, U and X represent, respectively, Female, Male, Unknown and Other.</br>
No 'Claim Injury Type' over 6 was attributed to genders other than M and F.

In [57]:
# we create a new dataframe with dummified 'Gender' and the target 
df_gender = pd.get_dummies(df, columns=['Gender'], prefix='Gender')
df_gender = df_gender[[col for col in df_gender.columns if col.startswith('Gender_')] + ['Claim Injury Type']]
df_gender['Claim Injury Type'] = df_gender['Claim Injury Type'].str[0]

In [58]:
# we check the correlation of the gender categories with the target
df_gender[[col for col in df_gender if col.startswith('Gender_')]].corrwith(df_gender['Claim Injury Type'])

Gender_F   -0.076225
Gender_M    0.081438
Gender_U   -0.029960
Gender_X    0.001268
dtype: float64

The categories themselves do not seem to have any correlation with the target and since we only have 46 values in category 'X', which is fairly a drop in the ocean considering we have almost 600k rows - it is best if we remove these values and this category itself.

In [59]:
df['Gender'].isna().sum()

19444

Since we have an 'Unknown' category ("U"), we will assign the missing values to this category.

In [60]:
df.loc[df['Gender'].isna(), 'Gender'] = 'U'

Now, as we want to delete the "X" category, we will set these values to NaN and later perform imputation.

In [61]:
df.loc[df['Gender'] == 'X'] = np.nan

In [62]:
df['Gender'].isna().sum()

46

### 7.5. Dates

In this section we will check the consistency between the date columns.

#### 7.5.1. Accident Date

In [63]:
len(df[
    (df['Accident Date'] > df['Assembly Date']) |  # accident happened after claim assembly
    (df['Accident Date'] > df['C-2 Date']) |  # accident happened after c-2 form delivery
    (df['Accident Date'] > df['C-3 Date']) |  # accident happened after c-3 form delivery
    (df['Accident Date'] > df['First Hearing Date'])  # accident happened after first hearing
])

1700

We don't have that many cases where 'Accident Date' is inconsistent - so we drop these rows.

In [64]:
accident_condition = (
    (df['Accident Date'] > df['Assembly Date']) |  # accident happened after claim assembly
    (df['Accident Date'] > df['C-2 Date']) |  # accident happened after c-2 form delivery
    (df['Accident Date'] > df['C-3 Date']) |  # accident happened after c-3 form delivery
    (df['Accident Date'] > df['First Hearing Date'])  # accident happened after first hearing
)

In [65]:
df = df.loc[accident_condition == False]

#### 7.5.2. Assembly Date

In [66]:
len(df[
    (df['Assembly Date'] > df['First Hearing Date']) |  # claim assembly happend after first hearing
    (df['Assembly Date'] < df['C-2 Date']) |  # claim was assembled before c-2 form delivery
    (df['Assembly Date'] < df['C-3 Date'])  # claim was assembled before c-3 form delivery
])

165258

In [67]:
len(df[
    (df['Assembly Date'] > df['First Hearing Date'])])

0

We have a lot of claims which have an inconsistency related with 'Assembly Date' - let's dive deeper into this.

##### 7.5.2.1. First Hearing Date

In [68]:
len(df[(df['Assembly Date'] > df['First Hearing Date'])])  # claim assembly happend after first hearing

0

##### 7.5.2.2. C-2 Date

In [69]:
len(df[(df['Assembly Date'] < df['C-2 Date'])])  # claim assembly happened after c-2 form delivery

67517

Switching these two columns' values should fix the problem - but let's see if there are cases where 'C-3 Date' is also after 'Assembly Date'

In [70]:
len(df[(df['Assembly Date'] < df['C-2 Date']) & (df['Assembly Date'] < df['C-3 Date'])]) # claim assembly happened after c-2 and c-3 form delivery

1262

There are few cases that have inconsistencies across the 3 columns - trying to fix this by making an arbitrary decision about which date should come first does not make much sense. </br>
Therefore, we drop these 1262 rows and fix the cases where there are inconsistencies only across two of the columns by switching them around.

In [71]:
assembly_condition = (
    (df['Assembly Date'] < df['C-2 Date']) &
    (df['Assembly Date'] < df['C-3 Date'])
)

In [72]:
# removing the rows with inconsistencies across the 3 columns
df = df.loc[assembly_condition == False]

In [73]:
c2_condition = (df['Assembly Date'] < df['C-2 Date'])  # claim assembly happened after c-2 form delivery

In [74]:
df.loc[c2_condition, ['Assembly Date', 'C-2 Date']] = df.loc[c2_condition, ['C-2 Date', 'Assembly Date']].values

##### 7.5.2.3. C-3 Date

In [75]:
len(df[(df['Assembly Date'] < df['C-3 Date'])])  # claim assembly happened after c-3 form delivery

97741

There are a lot of cases with inconsistent values, so we will also switch these two columns.

In [76]:
c3_condition = (df['Assembly Date'] < df['C-3 Date'])  # claim assembly happened after c-3 form delivery

In [77]:
df.loc[c3_condition, ['Assembly Date', 'C-3 Date']] = df.loc[c3_condition, ['C-3 Date', 'Assembly Date']].values

In [78]:
len(df[(df['Assembly Date'] < df['C-3 Date'])])  # claim assembly happened after c-3 form delivery

0

##### 7.5.2.4. Rechecking

In [79]:
# Recheck inconsistency
len(df[
    (df['Assembly Date'] > df['First Hearing Date']) |  # claim assembly happend after first hearing
    (df['Assembly Date'] < df['C-2 Date']) |  # claim was assembled before c-2 form delivery
    (df['Assembly Date'] < df['C-3 Date'])  # claim was assembled before c-3 form delivery
])

7061

In [80]:
len(df[(df['Assembly Date'] > df['First Hearing Date'])])

7061

In [81]:
hearing_condition = (df['Assembly Date'] > df['First Hearing Date'])

In [82]:
df.loc[hearing_condition, ['Assembly Date', 'First Hearing Date']] = df.loc[hearing_condition, ['First Hearing Date', 'Assembly Date']].values

In [83]:
# Recheck inconsistency
len(df[
    (df['Assembly Date'] > df['First Hearing Date']) |  # claim assembly happend after first hearing
    (df['Assembly Date'] < df['C-2 Date']) |  # claim was assembled before c-2 form delivery
    (df['Assembly Date'] < df['C-3 Date'])  # claim was assembled before c-3 form delivery
])

0

We have fixed the inconsistencies related with Assembly Date! Or so we taught...

In [84]:
len(df[df['Assembly Date'].dt.year > 2022])

7482

With all these transformations, we happened to create a new inconsistency: our dataset is only supposed to have data until the end of 2022 and we have some values that are after 2022. The best way to fix this is to cast this values to NaN and later impute them using an algorithm.

In [85]:
df.loc[df['Assembly Date'].dt.year > 2022, 'Assembly Date'] = np.nan

#### 7.5.3. First Hearing Date

In [86]:
len(df[
    (df['C-2 Date'] > df['First Hearing Date']) |  # c-2 form was delivered after first hearing
    (df['C-3 Date'] > df['First Hearing Date'])  # c-3 form was delivered after first hearing
])

0

All data is consistent regarding this feature.

#### 7.5.4. C-2 Date Timing

After a brief research, we know that the C-2 form must be submitted by the employer within 10 days of knowing about the accident.

In [87]:
len(df[df['C-2 Date'] > (df['Accident Date'] + timedelta(days=10))])

255654

Almost half of the data is inconsistent with the timing and so we will check the correlation of this inconsistency with the target variable.

In [88]:
c2_timing = df[df['C-2 Date'] > (df['Accident Date'] + timedelta(days=10))].copy()

In [89]:
c2_timing['Claim Injury Type'].value_counts(normalize=True)

Claim Injury Type
2. NON-COMP        0.481858
4. TEMPORARY       0.263755
3. MED ONLY        0.126746
5. PPD SCH LOSS    0.100444
1. CANCELLED       0.017363
6. PPD NSL         0.008265
8. DEATH           0.001357
7. PTD             0.000211
Name: proportion, dtype: float64

Around half of these values lead to a claim either cancelled or non-compliant. This tells us this information could potentially be a good target predictor.</br>
For that matter, we create a new binary variable 'C-2 Missed Timing' which tells us wether the C-2 form was delivered within the allowed timeframe or not.

In [90]:
df['C-2 Missed Timing'] = np.where(df['C-2 Date'] > (df['Accident Date'] + timedelta(days=10)), 1, 0)

#### 7.5.5. C-3 Date Timing

This form must be delivered within 2 years of the accident.

In [91]:
len(df[df['C-3 Date'] > (df['Accident Date'] + timedelta(days=(365*2)))])

1399

In [92]:
c3_timing = df[df['C-3 Date'] > (df['Accident Date'] + timedelta(days=(365*2)))].copy()

In [93]:
c3_timing['Claim Injury Type'].value_counts(normalize=True)

Claim Injury Type
2. NON-COMP        0.503931
5. PPD SCH LOSS    0.164403
4. TEMPORARY       0.144389
3. MED ONLY        0.102931
1. CANCELLED       0.077198
6. PPD NSL         0.005718
8. DEATH           0.000715
7. PTD             0.000715
Name: proportion, dtype: float64

Half of the claims that miss the C-3 form delivery timeline belong to the Non-compliant class - we will create new binary feature 'C-3 Missed Timing' which tells us if the C-3 form was delivered within the allowed timeframe.

In [94]:
df['C-3 Missed Timing'] = np.where(df['C-3 Date'] > (df['Accident Date'] + timedelta(days=(365*2))), 1, 0)

### 7.6. Covid-19 Indicator

In [95]:
df[
    (df['COVID-19 Indicator'] == "Y") &
    (df['Accident Date'].dt.year < 2020)
]

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing
6623,2019-12-17,29,N,2020-01-30,Y,2141.8,1990,2020-01-13,2020-01-08,"POLICE, FIRE, SANITATION",3A. SELF PUBLIC,5. PPD SCH LOSS,ROCKLAND,Y,NYC,NaT,F,3,92,PUBLIC ADMINISTRATION,III,NaN,83,PANDEMIC,83,COVID-19,48,INTERNAL ORGANS,10954,0,Not Work Related,3,1,0
192473,2019-12-22,53,N,2021-01-25,N,0.0,1966,2021-01-23,NaT,NYU LANGONE HOSPITALS,4A. SELF PRIVATE,2. NON-COMP,SUFFOLK,Y,HAUPPAUGE,NaT,M,<NA>,62,HEALTH CARE AND SOCIAL ASSISTANCE,IV,NaN,83,PANDEMIC,83,COVID-19,91,BODY SYSTEMS AND MULTIPLE BODY SYSTEMS,11787,0,Not Work Related,0,1,0


Seems like everything is consistent regarding Covid-19 claims

### 7.7. The wanna-be NaN

As we have seen in [Chapter 6](#6-unique-values), we have string representations of missing values in 'Zip Code' - we shall convert these to the default NaN from numpy.

In [96]:
df['Zip Code'] = df['Zip Code'].replace('nan', np.nan)

### 7.8. Duplicated values

In [97]:
# we check how many duplicated rows we have
df.duplicated(keep=False).sum()

19492

In [98]:
# we check how many of these are distinct
len(df[df.duplicated(keep=False)].drop_duplicates())

1098

In fact, even though we have 19k+ duplicated rows, 1k of these rows are distinct, i.e., we have rows with more than one duplicate

In [99]:
df[df.duplicated()].drop_duplicates().head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing
33,NaT,<NA>,NaN,2020-01-01,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
188,NaT,<NA>,NaN,2020-01-02,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
976,NaT,<NA>,NaN,2020-01-03,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
1552,NaT,<NA>,NaN,2020-01-04,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
1573,NaT,<NA>,NaN,2020-01-05,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0


We shall check if all values of all rows are NaN with the exception of 'Assembly Date' and the newly created column 'C-2 Missed Timing'

In [100]:
# drop duplicates from those duplicated rows
unique_duplicated_rows = df[df.duplicated(keep=False)].drop_duplicates()

# check if all values (excluding 'Assembly Date', 'C-2 Missed Timing' and 'C-3 Missed Timing') are NaN
nan_check = unique_duplicated_rows.drop(columns=['Assembly Date', 'C-2 Missed Timing', 'C-3 Missed Timing']).isna().all(axis=1)

# we check the rows that have other than missing values and gender different from "U"
unique_duplicated_rows[~nan_check].query("Gender != 'U'")

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing
226843,NaT,0,N,2021-03-25,Y,0.0,1968,NaT,2021-03-16,ACE AMERICAN INSURANCE CO,1A. PRIVATE,1. CANCELLED,UNKNOWN,N,NYC,2022-10-24,M,<NA>,<NA>,NaN,UK,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,07032,0,Not Work Related,0,0,0


This means that all duplicated rows have missing values in every column but 'Assembly Date', except for one row

In [101]:
# percentage of duplicate values
df.duplicated().sum() / df.shape[0] * 100

3.1253557526153655

The duplicated (and "empty") rows we have been discussing account for around 3.1% of our data - we will drop these values.

In [102]:
df.drop_duplicates(inplace=True)

### 7.9. The target

In [103]:
# we check how many missing values we have in our target variable
df['Claim Injury Type'].isna().sum()

1097

In [104]:
df[df['Claim Injury Type'].isna()].head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing
3,NaT,<NA>,NaN,2020-01-01,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
180,NaT,<NA>,NaN,2020-01-02,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
792,NaT,<NA>,NaN,2020-01-03,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
1551,NaT,<NA>,NaN,2020-01-04,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0
1572,NaT,<NA>,NaN,2020-01-05,NaN,NaN,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,U,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,0,0


At a first glance, it looks like these rows are all NaN values with the exception of 'Assembly Date' and the new column and with gender as "U" (Unknown) - just like before. Let us check if this is indeed the case.

In [105]:
no_target_df = df[df['Claim Injury Type'].isna()]

# we check if all values (excluding 'Assembly Date', 'C-2 Missed Timing' and 'C-3 Missed Timing') are NaN
nan_check = no_target_df.drop(columns=['Assembly Date', 'C-2 Missed Timing', 'C-3 Missed Timing']).isna().all(axis=1)

# we check the rows that have other than missing values and gender different from "U"
no_target_df[~nan_check].query("Gender != 'U'")

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing


It seems like all these rows have missing values in every column but 'Assembly Date' - we will also remove these rows.

In [106]:
df = df[~df['Claim Injury Type'].isna()]

Knowing the original dataset had 593471 rows, let us check what percentage of the data we have removed so far.

In [107]:
(1 - (df.shape[0] / 593471)) * 100

4.114944116898722

We are still below the 5% threshold, so we are good to go!

## 8. Feature Engineering

In [108]:
# amount of days between the accident and the claim assembly
df['Days Difference'] = (df['Assembly Date'] - df['Accident Date']).dt.days

# binary columns if the date is missing or not
df['C-2 Missing'] = df['C-2 Date'].isnull().astype(int)
df['C-3 Missing'] = df['C-3 Date'].isnull().astype(int)
df['Has Hearing'] = df['First Hearing Date'].isnull().astype(int)
df['Has IME-4 Report'] = np.where(df['IME-4 Count'].notna(), 1, 0)

In [109]:
df.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents,C-2 Missed Timing,C-3 Missed Timing,Days Difference,C-2 Missing,C-3 Missing,Has Hearing,Has IME-4 Report
0,2019-12-30,31,N,2020-01-01,N,0.00,1988,2019-12-31,NaT,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaT,M,<NA>,44,RETAIL TRADE,I,NaN,27,FROM LIQUID OR GREASE SPILLS,10,CONTUSION,62,BUTTOCKS,13662,0,Not Work Related,1,0,0,2.0,0,1,1,0
1,2019-08-30,46,N,2020-01-14,Y,1745.93,1973,2020-01-01,2020-01-01,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,2020-02-21,F,4,23,CONSTRUCTION,I,NaN,94,REPETITIVE MOTION,49,SPRAIN OR TEAR,38,SHOULDER(S),14569,1,Not Work Related,4,1,0,137.0,0,0,0,1
2,2019-12-06,40,N,2020-01-01,N,1434.80,1979,2020-01-01,NaT,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaT,M,<NA>,56,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIAT,II,NaN,17,OBJECT BEING LIFTED OR HANDLED,7,CONCUSSION,10,MULTIPLE HEAD INJURY,12589,0,Not Work Related,6,1,0,26.0,0,1,1,0
4,2019-12-30,61,N,2020-01-01,N,NaN,1958,2019-12-31,NaT,STATE INSURANCE FUND,2A. SIF,2. NON-COMP,DUTCHESS,N,ALBANY,NaT,M,<NA>,62,HEALTH CARE AND SOCIAL ASSISTANCE,II,NaN,16,"HAND TOOL, UTENSIL; NOT POWERED",43,PUNCTURE,36,FINGER(S),12603,0,Not Work Related,1,0,0,2.0,0,1,1,0
5,2019-12-26,67,N,2020-01-01,N,0.00,1952,2019-12-31,NaT,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,3. MED ONLY,SUFFOLK,N,HAUPPAUGE,NaT,M,<NA>,44,RETAIL TRADE,IV,NaN,31,"FALL, SLIP OR TRIP, NOC",10,CONTUSION,38,SHOULDER(S),11772,0,Not Work Related,5,0,0,6.0,0,1,1,0


## 9. Data Statistics

Let us recheck our count of missing values now that we have fixed some inconsistencies and look at some summary statistics

### 9.1. Missing values (again)

In [110]:
# Calculate the number of NaNs for each column
nan_counts = df.isna().sum()

# Get the total number of rows (entries) in the DataFrame
total_rows = df.shape[0]

# Calculate the percentage of NaN values for each column
percentage_nans = (nan_counts / total_rows) * 100

# Format the percentage with '%' sign
percentage_nans = percentage_nans.apply(lambda x: f"{x:.2f}%")

# Combine all information into a DataFrame for better readability
nan_summary = pd.DataFrame({
    'NaN Count': nan_counts,
    'Total Values': [total_rows] * len(nan_counts),  # Ensure this column matches the length of nan_counts
    'Percentage NaN': percentage_nans
})

# Print the result
print("Summary of NaN values per column:\n")
print(nan_summary)

Summary of NaN values per column:

                                    NaN Count  Total Values Percentage NaN
Accident Date                            3347        569050          0.59%
Age at Injury                               0        569050          0.00%
Alternative Dispute Resolution              0        569050          0.00%
Assembly Date                            7482        569050          1.31%
Attorney/Representative                     0        569050          0.00%
Average Weekly Wage                     28404        569050          4.99%
Birth Year                              28929        569050          5.08%
C-2 Date                                13602        569050          2.39%
C-3 Date                               385183        569050         67.69%
Carrier Name                                0        569050          0.00%
Carrier Type                                0        569050          0.00%
Claim Injury Type                           0        569050      

We have considerably reduced the number of missing values - nice!

### 9.2. Summary statistics

In [111]:
df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
Accident Date,565703,NaN,NaN,NaN,2021-05-01 18:57:06.924729088,1961-09-06 00:00:00,2020-09-15 00:00:00,2021-06-28 00:00:00,2022-03-21 00:00:00,2022-12-30 00:00:00,NaN
Age at Injury,569050.0,<NA>,<NA>,<NA>,42.243149,0.0,31.0,42.0,54.0,120.0,14.089729
Alternative Dispute Resolution,569050,3,N,566449,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Assembly Date,561568,NaN,NaN,NaN,2021-07-29 06:30:39.950994432,2020-01-01 00:00:00,2020-11-12 00:00:00,2021-08-11 00:00:00,2022-04-26 00:00:00,2022-12-31 00:00:00,NaN
Attorney/Representative,569050,2,N,390596,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Average Weekly Wage,540646.0,NaN,NaN,NaN,488.712178,0.0,0.0,0.0,836.5,2828079.0,6118.795043
Birth Year,540121.0,<NA>,<NA>,<NA>,1977.827461,1900.0,1967.0,1978.0,1989.0,2008.0,13.848329
C-2 Date,555448,NaN,NaN,NaN,2021-07-09 17:07:08.333165824,1996-01-12 00:00:00,2020-10-27 00:00:00,2021-07-30 00:00:00,2022-04-20 00:00:00,2022-12-31 00:00:00,NaN
C-3 Date,183867,NaN,NaN,NaN,2021-06-20 16:37:24.735596800,2010-05-14 00:00:00,2020-09-29 00:00:00,2021-06-17 00:00:00,2022-03-15 00:00:00,2022-12-30 00:00:00,NaN
Carrier Name,569050,2039,STATE INSURANCE FUND,110414,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- **Accident Date**: more than 75% of accidents happened after 2020, eventhough the first accidents happened in 1961
- **Age at Injury**: the average age at the time of injury is 42 years-old; we have very low values (which we will fix after imputation of other columns) and very high values (a brief search online tells us that the average retirement age is between 63 and 65, maybe values much larger than these do not make much sense)
- **Attorney/Representative**: most claims do not have an attorney/representative
- **Average Weekly Wage**: more than half of the people in the dataset have 0 as their average weekly wage; the highest earning person earns more than $2.8B a week
- **Gender**: male are more prone to have a workplace injury
- **IME-4 Count**: on average, a claim gets 3 examiner reports, but the most reports a claim has gotten was 73 (this may be a bit unusual)
- **Industry Code Description**: the industry which gets more claims is HEALTH CARE AND SOCIAL ASSISTANCE
- **WCIO Cause of Injury Description**: LIFTING is the biggest cause of injuries
- **WCIO Nature of Injury Description**: STRAIN OR TEAR is the biggest nature of injuries
- **WCIO Part Of Body Description**: people most get injured on their LOWER BACK AREA
- **Agreement Reached**: most claims need the involvement of the WCB to get closed
- **Number of Dependents**: people have on average 3 dependents
- **C-2 Missed Timing**: the C-2 form delivery date is missed more than half the times
- **C-3 Missed Timing**: the C-3 form delivery date is missed more than 75% of the times
- **Days Difference**: on average, it takes 3 months for the claim to be assembled; however, we could have some extreme values as the person in the 75% percentile on needed 40 days to get their claim assembled
- **C-2 Missing**: more than 75% of claims have the C-2 form missing
- **Has IME-4 Report**: more than 75% of claims do not have an examiner report

## 10. Export

We export the data with the already made changes and will continue our analysis in the notebook 'Part2-VisualExploration.ipynb'

In [112]:
df.to_csv('../Data/train_data_initial_inspection.csv', index=True)